In [22]:
import os
import json
import pandas as pd
from sqlalchemy import create_engine

# Configuration
JSON_FILE = 'resources/insurance_claims.json'                # Path to your JSON data file
RESOURCES_DIR = 'resources'               # Folder to store the database
DB_FILENAME = 'insurance_claims.db'      # Database file name
TABLE_NAME = 'claims'

# Ensure resources directory exists
os.makedirs(RESOURCES_DIR, exist_ok=True)
# Construct full database path
db_path = os.path.join(RESOURCES_DIR, DB_FILENAME)
# SQLite database URI
DATABASE_URI = f'sqlite:///{db_path}'


def load_data_from_json(filepath: str) -> pd.DataFrame:
    """
    Load claim records from a JSON file into a pandas DataFrame.
    The JSON should be a list of objects matching the schema.
    """
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
    df = pd.DataFrame(data)
    # Convert date strings to datetime
    if 'Date of Loss' in df.columns:
        df['Date of Loss'] = pd.to_datetime(df['Date of Loss'])
    return df


def save_to_sql(df: pd.DataFrame, db_uri: str, table_name: str):
    """
    Save the DataFrame to the specified SQL table.
    Replaces the table if it already exists.
    """
    engine = create_engine(db_uri)
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    print(f"Inserted {len(df)} records into '{table_name}' in database at '{db_uri}'")

In [23]:
df_claims = load_data_from_json(JSON_FILE)

# Save to database
save_to_sql(df_claims, DATABASE_URI, TABLE_NAME)


Inserted 19 records into 'claims' in database at 'sqlite:///resources\insurance_claims.db'
